In [64]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import SGD
from keras.utils import to_categorical
from keras.models import Sequential
from time import time
from keras.layers import Dense, Conv2D, Flatten

In [154]:
# Import dataset
train = pd.read_csv('optdigits.tra', header=None)
test = pd.read_csv('optdigits.tes', header=None)

In [155]:
# Sort x and y
train_x = train.iloc[:, 0:64]
train_y = train[64]
test_x = test.iloc[:, 0:64]
test_y = test[64]

In [156]:
# Standardization
train_x = train_x.astype('float32') / 16.
test_x = test_x.astype('float32') / 16.

In [157]:
# Split training set into training set and validation set
new_train_x, val_x, new_train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=1)

In [158]:
# Changed dataset into numpy
new_train_x = new_train_x.to_numpy()
new_train_y = new_train_y.to_numpy()
val_x = val_x.to_numpy()
val_y = val_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()

In [159]:
# Change the shape of y data set
conv_train_x = new_train_x.reshape(new_train_x.shape[0],8,8,1)
conv_test_x = test_x.reshape(test_x.shape[0],8,8,1)
conv_val_x = val_x.reshape(val_x.shape[0],8,8,1)
conv_train_y = to_categorical(new_train_y)
conv_test_y = to_categorical(test_y)
conv_val_y = to_categorical(val_y)

In [160]:
# Creat a model
model = Sequential()
model.add(Conv2D(15, kernel_size=(5,5), activation= 'relu', input_shape=(8,8,1)))
model.add(Flatten())
model.add(Dense(10, activation= 'softmax'))


In [161]:
# Set SGD as the optimizer
opt = SGD(learning_rate=0.2, momentum=0.2, nesterov=False)

In [162]:
# Early Stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                              min_delta=0, patience=0, 
                              verbose=0, mode='auto')

In [163]:
# Model Compiling
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [164]:
# Fit Model
start = time()
model.fit(conv_train_x, conv_train_y, epochs= 150, batch_size=15,
          callbacks = [early_stop], validation_data=(conv_val_x, conv_val_y))
print(time()-start)

Train on 3058 samples, validate on 765 samples
Epoch 1/150
3058/3058 [==============================] - 0s 153us/step - loss: 0.5117 - accuracy: 0.8473 - val_loss: 0.2791 - val_accuracy: 0.9216
Epoch 2/150
3058/3058 [==============================] - 0s 132us/step - loss: 0.1612 - accuracy: 0.9493 - val_loss: 0.2046 - val_accuracy: 0.9438
Epoch 3/150
3058/3058 [==============================] - 0s 137us/step - loss: 0.1259 - accuracy: 0.9595 - val_loss: 0.1375 - val_accuracy: 0.9556
Epoch 4/150
3058/3058 [==============================] - 0s 146us/step - loss: 0.0995 - accuracy: 0.9709 - val_loss: 0.1168 - val_accuracy: 0.9673
Epoch 5/150
3058/3058 [==============================] - 0s 129us/step - loss: 0.0828 - accuracy: 0.9725 - val_loss: 0.1201 - val_accuracy: 0.9660
2.481171131134033


In [165]:
# Distribution of training set  >>> counted with for loops
dist_train = [376,389,380,389,387,376,377,387,380,382]
# Distribution of testing set
dist_test = [178,182,177,183,181,182,181,179,174,180]

In [166]:
# Accuracay of overall classification accuracy and class accuracy
def accuracy_overall(pred, actual, num_classes):
    result = int()
# Overall classfication accuracy
    overall = np.where(pred == actual)[0]
    overall_acc = float(len(overall))/len(pred)
    result = overall_acc
    return result

def accuracy(pred, actual, num_classes):
# Class accuracy
    result = []
    classes = np.array([float(0)]*10)
    for i in range(0,len(pred)):
        id_classes = int(actual[i])
        if pred[i] == actual[i]:
            classes[id_classes] = classes[id_classes] + 1
    classes_acc = classes/num_classes
    result.append(classes_acc)
    return result

In [167]:
# confusion matrix
def confusion_matrix(pred, actual):
    matrix = np.array([int(0)]*10*10).reshape(10,10)
    for i in range(0,10):
        pred_i = pred[np.where(np.array(actual) == i)]
        conf = []
        for j in range(0,10):
            length = len(np.where(pred_i == j)[0])
            conf.append(length)
        matrix[i,] = conf
    return matrix

In [168]:
test_y = test[64]
test_y = test_y.to_numpy()

In [178]:
train_x = train_x.to_numpy()
train_x = train_x.reshape(train_x.shape[0],8,8,1)
test_x = test_x.reshape(test_x.shape[0],8,8,1)

In [179]:
predict_train = model.predict(train_x)
predict_train_class = predict_train.argmax(axis=-1)
acc_train_overall = accuracy_overall(predict_train_class, train_y, dist_train)
acc_train_class = accuracy(predict_train_class, train_y, dist_train)
conf_train = confusion_matrix(predict_train_class, train_y)


predict_test = model.predict(test_x)
predict_test_class = predict_test.argmax(axis=-1)
acc_test_overall = accuracy_overall(predict_test_class, test_y, dist_test)
acc_test_class = accuracy(predict_test_class, test_y, dist_test)
conf_test = confusion_matrix(predict_test_class, test_y)

In [180]:
#output results
print("training set overall accuracy" + "\n")
print(acc_train_overall)
print("\n")
print("training set class accuracy" + "\n")
print(acc_train_class)
print("\n")
print("training set confusion matrix" + "\n")
print(conf_train)
print("\n")
print("testing set overall accuracy" + "\n")
print(acc_test_overall)
print("\n")
print("testing set class accuracy" + "\n")
print(acc_test_class)
print("\n")
print("testing set confusion matrix" + "\n")
print(conf_test)

training set overall accuracy

0.9816897724300288


training set class accuracy

[array([0.99468085, 0.98457584, 0.98947368, 0.96143959, 0.97674419,
       0.98138298, 0.99204244, 0.99741602, 0.98421053, 0.95549738])]


training set confusion matrix

[[374   0   0   0   1   0   0   0   1   0]
 [  0 383   0   0   0   0   0   1   3   2]
 [  0   1 376   0   0   0   0   1   1   1]
 [  0   0   1 374   0   4   0   1   5   4]
 [  0   0   0   0 378   0   2   0   0   7]
 [  0   1   2   1   0 369   0   0   0   3]
 [  0   2   0   0   1   0 374   0   0   0]
 [  0   0   0   1   0   0   0 386   0   0]
 [  0   3   0   0   1   0   1   0 374   1]
 [  0   5   2   2   2   1   0   1   4 365]]


testing set overall accuracy

0.9571508069003896


testing set class accuracy

[array([0.99438202, 0.96703297, 0.97740113, 0.92349727, 0.95027624,
       0.97802198, 0.97790055, 0.92178771, 0.93103448, 0.95      ])]


testing set confusion matrix

[[177   0   0   0   1   0   0   0   0   0]
 [  0 176   0   0   0   0